# <a name="1common."></a>
## <a name="0.0"></a>Содержание:
* [1. Загрузка данных и библиотек](#0.)
* [2. Подготовка данных](#8.)
* [3. Классификация полученных изображений](#10.)
* [4. CLIP](#16.)
* [5. Flask Вывод запроса и изображения](#17.)

## 1. Анализ данных
<a name="0."></a>
[<font size="2">(к содержанию)</font>](#1common.)

In [1]:
pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-g6nansc0
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-g6nansc0
     |████████████████████████████████| 71kB 3.1MB/s 
     |████████████████████████████████| 776.8MB 25kB/s 
     |████████████████████████████████| 12.8MB 239kB/s 
  Created wheel for clip: filename=clip-1.0-cp37-none-any.whl size=1368688 sha256=3d60e721d60fe5a6f4daaaaa1ab4b72ac2432b11c9ae6c5415cb21831bfa25e8
  Stored in directory: /tmp/pip-ephem-wheel-cache-6swesmdp/wheels/79/51/d7/69f91d37121befe21d9c52332e04f592e17d1cabc7319b3e09
  Created wheel for ftfy: filename=ftfy-5.9-cp37-none-any.whl size=46451 sha256=e91baae711d3c9c2b86e89e2d01fd892e1d8ec2e8dd30d90d75ea9a34939449a
  Stored in directory: /root/.cache/pip/wheels/5e/2e/f0/b07196e8c929114998f0316894a61c752b63bfa3fdd50d2fc3
Successfully built clip ftfy
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.7.1 which is incompatible.
  Foun

In [2]:
#загрузим данные
!pip install gdown==3.6.0
!pip install gdown==3.6.0
my_file_id = "1V_x21M0dNb9pDUz3RbogAeau877oU4Z3"
!gdown https://drive.google.com/uc?id={my_file_id}

  Created wheel for gdown: filename=gdown-3.6.0-cp37-none-any.whl size=5237 sha256=bf485416498bca0b4e6a962de2cf3c0bc3e5da862d269d8315542ddecf734546
  Stored in directory: /root/.cache/pip/wheels/97/90/fa/25654eb65da3e6da7752db71a164e0eb8f7a6fb4335eeb46ab
Successfully built gdown
  Found existing installation: gdown 3.6.4
    Uninstalling gdown-3.6.4:
      Successfully uninstalled gdown-3.6.4
Downloading...
From: https://drive.google.com/uc?id=1V_x21M0dNb9pDUz3RbogAeau877oU4Z3
To: /content/new_f-img.zip
63.1MB [00:00, 181MB/s] 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!mkdir /content/img/
!unzip -qq /content/new_f-img.zip -d /content/img/

In [14]:
#загрузка модели
my_file_id = "1fKf7ciCwBWIsIjlnnpE4hy7Dl7dtipv6"
!gdown https://drive.google.com/uc?id={my_file_id}

Downloading...
From: https://drive.google.com/uc?id=1fKf7ciCwBWIsIjlnnpE4hy7Dl7dtipv6
To: /content/model_CFT_face
44.8MB [00:00, 230MB/s]


In [5]:
!pip install opencv-python

In [6]:
#библиотеки
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.preprocessing.image import load_img, img_to_array
from google.colab.patches import cv2_imshow
import cv2
import re
import shutil
import torch
import torchvision
from torchvision import datasets, models, transforms

In [7]:
print(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))

cuda:0


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
#ссылка на директорию с фото
IMAGES_PATH = '/content/img/new_f-img'

In [17]:
#загрузим модель для определения трех классов i, bro, и other
model_face = torch.load('/content/drive/MyDrive/model/model_CFT_face', device)

## 2. Подготовка данных;
<a name="9."></a>
[<font size="2">(к содержанию)</font>](#1common.)

In [11]:
#словарь название изображения - подпись под изображением
images_labels_dict = {}

for image in os.listdir(IMAGES_PATH):
  image_path = os.path.join(IMAGES_PATH, image)
  #clean labels from filename
  image_label = image.replace(".jpg", '')
  image_label = image_label.replace(".JPG", '')
  image_label = image_label.replace(".png", '')
  image_label = image_label.replace(".PNG", '')
  image_label = image_label.split("_")
  images_labels_dict.update({image_path:image_label})


In [12]:
#предсказание классов
def predict(image, model):

    output = model.forward(image)

    output = torch.exp(output)

    probs, classes = output.topk(1, dim=1)
    return probs.item(), classes.item()

## 3. Классификация полученных изображений;
<a name="10."></a>
[<font size="2">(к содержанию)</font>](#1common.)

In [13]:
#выполнение нахождения лица на фотографии с последующим его определением к одному из двух классов
face_cascade_db = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
!mkdir /content/img_face/

def face_img(image_names, num):
  !rm -rf /content/img_face/.ipynb_checkpoints
  image_i = []
  for i in image_names:

    img = cv2.imread(i)
    img_name = i
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade_db.\
        detectMultiScale(img, 1.1, 19)        
    for (x,y,w,h) in faces:
        roi_color = img[y-30:y+60+h, x-60:x+60+w]
        cv2.imwrite(r'/content/img_face/{}.jpg'.format(x), roi_color)
        IMAGES_PATH_2 = '/content/img_face/'
        name_image = []
        for image in os.listdir(IMAGES_PATH_2):
          name_image.append(os.path.join(IMAGES_PATH_2,image))
          
        photos = [Image.open(photo_file) for photo_file in name_image] 
        photos_preprocessed = [preprocess(photo) for photo in photos]
        for i in photos_preprocessed:
          img_top = i[np.newaxis,:].to(device)         
          top_prob, top_class = predict(img_top, model_face)
          if num == 0:
            if top_class == 0:
              image_i.append(img_name)
          if num == 1:
            if top_class == 1:
              image_i.append(img_name)
  #очистим папку
  for filename in os.listdir('/content/img_face/'):
    os.unlink('/content/img_face/' + filename)
  return image_i

## 4. CLIP
<a name="16."></a>
[<font size="2">(к содержанию)</font>](#1common.)

In [18]:
#CLIP
import clip
import torch
!mkdir static


# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)


# Prepare the inputs
#аргументы - текстовый запрос и количество желаемых фотографий

def find_picture(inputs, numbrr):
  #чистим папку static
  for filename in os.listdir('/content/static/'):
    os.unlink('/content/static/' + filename)

  name_image = []

  for image in os.listdir(IMAGES_PATH):
    name_image.append(os.path.join(IMAGES_PATH,image))

  photos = [Image.open(photo_file) for photo_file in name_image]
      
  photos_preprocessed = torch.stack([preprocess(photo) for photo in photos]).to(device)

  user_input = inputs
  find_my = 0
  #условие для запуска поиска по лицу i или bro
  if re.search(r'\bi\b', user_input):
    find_my += 1
  if re.search(r'\bbro\b', user_input):
    find_my -= 2


  user_input = user_input.replace("i", 'man')
  #print('='*60)
  text_inputs = clip.tokenize(user_input).to(device)

  # Calculate features
  with torch.no_grad():
      image_features = model.encode_image(photos_preprocessed)
      text_features = model.encode_text(text_inputs)

  # Pick the top 5 most similar labels for the image
  image_features /= image_features.norm(dim=-1, keepdim=True)
  text_features /= text_features.norm(dim=-1, keepdim=True)
  similarity = (100.0 * text_features @ image_features.T).softmax(dim=-1)
  values, indices = similarity[0].topk(numbrr)


  IMAGE_FOLDER = []

  
  #поиcк i или bro по условию
  if find_my > 0:
    IMAGE_FOLDER = []
    for value, index in zip(values, indices):
      IMAGE_FOLDER.append(name_image[index])

    IMAGE_FOLDER = face_img(IMAGE_FOLDER, 0)
 

  if find_my < 0:
    IMAGE_FOLDER = []
    for value, index in zip(values, indices):
      IMAGE_FOLDER.append(name_image[index])

    IMAGE_FOLDER = face_img(IMAGE_FOLDER, 1)

    
  #вывод по названию фото
  else:
    for image, label in images_labels_dict.items():
          if user_input in label:
            #print(f"All pictures with {user_input}")
            image = Image.open(image)
            IMAGE_FOLDER.append(name_image[index])
            #plt.imshow(image)
            #plt.show()
    print('='*60)


    # Print the result
    #print("\nTop predictions for %s:\n"%(user_input))
    for value, index in zip(values, indices):
        #print(f"{name_image[index]:>16s}: {100 * value.item():.2f}%")
        image = Image.open(name_image[index])
        IMAGE_FOLDER.append(name_image[index])
        #plt.imshow(image)
        #plt.show()




  #вывод для Flask в папку static
  for i in IMAGE_FOLDER:

    dst = '/content/static/'
    shutil.copy(i, dst)

 
  IMAGES_PATH_filt = '/content/static'

  name_image_filt = []


  for image in os.listdir(IMAGES_PATH_filt):
    name_image_filt.append(image)

  return name_image_filt

100%|████████████████████████████████████████| 354M/354M [00:02<00:00, 170MiB/s]


In [ ]:
#!rm -rf /content/img_face/.ipynb_checkpoints

## 5. Flask Вывод запроса и изображения
<a name="17."></a>
[<font size="2">(к содержанию)</font>](#1common.)

In [19]:
# installation
!pip install flask==0.12.2
!pip install flask-ngrok

     |████████████████████████████████| 92kB 3.7MB/s 
  Found existing installation: Flask 1.1.2
    Uninstalling Flask-1.1.2:
      Successfully uninstalled Flask-1.1.2


In [20]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, session, request, redirect 

In [21]:

!mkdir templates
%cd /content/templates

html = """
 <!DOCTYPE html>
<html>
<head>
    <title> Page </title>
    <meta http-equiv="Content-type" content="text/html">
</head>
<body>
    <form action="{{ url_for("page_start") }}" method="post">
        <p><b>Hello!</b><br>
        <p><b>Please enter a word to search: </b><br>
        <input name = "search_word" type = 'text' autofocus placeholder = "here">
	<p><b>How many pictures? </b><br>
	<input name = "pictures_amount" type="number" autofocus placeholder = "here" size="2" min="1" max="99" value="1">
        <p> <input type = "submit" value = "Send"> </p>
</form>
</body>
</html>"""

with open("index.html","a+") as f:
  f.write(html)

html_pred = """
<!DOCTYPE html>
<html>
<head>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Page_start</title>
<style>
* {
  box-sizing: border-box;
}

body {
  margin: 0;
  font-family: Arial, Helvetica, sans-serif;
}

.header {
  text-align: center;
  padding: 32px;
}

.row {
  display: -ms-flexbox; /* IE 10 */
  display: flex;
  -ms-flex-wrap: wrap; /* IE 10 */
  flex-wrap: wrap;
  padding: 0 4px;
}

/* Create two equal columns that sits next to each other */
.column {
  -ms-flex: 50%; /* IE 10 */
  flex: 50%;
  padding: 0 4px;
}

.column img {
  margin-top: 8px;
  vertical-align: middle;
}

/* Style the button */
.btn {
  border: none;
  outline: none;
  padding: 10px 16px;
  background-color: #f1f1f1;
  cursor: pointer;
  font-size: 18px;
}

.btn:hover {
  background-color: #ddd;
}

.btn.active {
  background-color: #666;
  color: white;
}
</style>
</head>
<body>
    <div class="row"> 
        <div class="column">
    {% for name in images %}
    <img src="{{url_for('static', filename=name)}}" style="width:20%" />
  {% endfor %}
  </div>
</div>

</body>
</html>"""

with open("page_start.html","a+") as f1:
  f1.write(html_pred)

%cd /content

/content/templates
/content


In [ ]:
#Fkask
template_dir = os.path.abspath('/content/templates')
app = Flask(__name__, template_folder = template_dir) 
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route('/', methods=['POST', 'GET'])
def home():
    return render_template('index.html')

@app.route("/page_start", methods = ['POST'])
def page_start():
    input = request.form['search_word']
    number = int(request.form['pictures_amount'])
    images = find_picture(input, number)

    return render_template('page_start.html', images = images)
    
app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://84eb01649e16.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
